# SQL-Based Churn KPI Analysis


In [6]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("../churn.db")


The dataset was loaded into a SQLite database to simulate real-world data storage used in production environments.


In [7]:
schema = pd.read_sql("PRAGMA table_info(customers);", conn)
schema

,cid,name,type,notnull,dflt_value,pk
0,0,customerID,TEXT,0,None,0
1,1,gender,TEXT,0,None,0
2,2,SeniorCitizen,INTEGER,0,None,0
3,3,Partner,TEXT,0,None,0
4,4,Dependents,TEXT,0,None,0
5,5,tenure,INTEGER,0,None,0
6,6,PhoneService,TEXT,0,None,0
7,7,MultipleLines,TEXT,0,None,0
8,8,InternetService,TEXT,0,None,0
9,9,OnlineSecurity,TEXT,0,None,0


## Column Categorization

Identifiers:
- customerID → Unique customer key

Demographic:
- gender
- SeniorCitizen
- Partner
- Dependents

Service-related:
- PhoneService
- InternetService
- OnlineSecurity
- OnlineBackup
- DeviceProtection
- TechSupport
- StreamingTV
- StreamingMovies

Account-related:
- tenure
- Contract
- PaperlessBilling
- PaymentMethod

Financial:
- MonthlyCharges
- TotalCharges

Target:
- Churn


In [13]:
pd.read_sql("""
SELECT gender, COUNT(*) AS count
FROM customers
GROUP BY gender;
""", conn)


,gender,count
0,Female,3483
1,Male,3549


In [8]:
pd.read_sql("SELECT * FROM customers LIMIT 5;", conn)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [14]:
pd.read_sql("""
SELECT Contract, COUNT(*) AS count
FROM customers
GROUP BY Contract;
""", conn)


,Contract,count
0,Month-to-month,3875
1,One year,1472
2,Two year,1685


In [15]:
pd.read_sql("""
SELECT Churn, COUNT(*) as count
FROM customers
GROUP BY Churn;
""", conn)


,Churn,count
0,No,5163
1,Yes,1869


Approximately 26–27% of customers have churned.
This indicates moderate churn risk in the customer base.
Given that churn represents revenue loss, identifying high-risk customers is critical for retention strategy.
Due to class imbalance, accuracy alone is not a sufficient evaluation metric. 
Recall for the churn class will be prioritized.


In [16]:
pd.read_sql("""
SELECT InternetService, COUNT(*) as count
FROM customers
GROUP BY InternetService;
""", conn)


,InternetService,count
0,DSL,2416
1,Fiber optic,3096
2,No,1520


In [17]:
pd.read_sql("""
SELECT MIN(tenure), MAX(tenure), AVG(tenure)
FROM customers;
""", conn)


,MIN(tenure),MAX(tenure),AVG(tenure)
0,1,72,32.421786


Customer tenure ranges from 1 to 72 months, with an average of approximately 32 months.

This suggests:
- A mix of new and long-term customers.
- Customer loyalty likely increases with tenure.
- Early-stage customers may exhibit higher churn risk.


In [18]:
pd.read_sql("""
SELECT MIN(MonthlyCharges), MAX(MonthlyCharges), AVG(MonthlyCharges)
FROM customers;
""", conn)


,MIN(MonthlyCharges),MAX(MonthlyCharges),AVG(MonthlyCharges)
0,18.25,118.75,64.798208


Monthly charges range between €18.25 and €118.75, with an average of approximately €64.80.

This indicates:
- A wide range of subscription plans.
- Potential segmentation between low-tier and premium customers.
- Higher-paying customers may contribute disproportionately to revenue loss if they churn.


### Overall Churn Rate

In [10]:
query = """
SELECT 
    COUNT(*) AS total_customers,
    SUM(CASE WHEN Churn='Yes' THEN 1 ELSE 0 END) AS churned_customers,
    ROUND(
        100.0 * SUM(CASE WHEN Churn='Yes' THEN 1 ELSE 0 END) / COUNT(*), 2
    ) AS churn_rate_percentage
FROM customers;
"""

pd.read_sql(query, conn)


,total_customers,churned_customers,churn_rate_percentage
0,7032,1869,26.58


Approximately 26.54% of customers have churned.
This indicates moderate churn risk within the subscription base.


### Churn by Contract Type

In [19]:
pd.read_sql("""
SELECT 
    Contract,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN Churn='Yes' THEN 1 ELSE 0 END) AS churned,
    ROUND(
        100.0 * SUM(CASE WHEN Churn='Yes' THEN 1 ELSE 0 END) / COUNT(*), 2
    ) AS churn_rate
FROM customers
GROUP BY Contract
ORDER BY churn_rate DESC;
""", conn)


,Contract,total_customers,churned,churn_rate
0,Month-to-month,3875,1655,42.71
1,One year,1472,166,11.28
2,Two year,1685,48,2.85


Customers on month-to-month contracts exhibit significantly higher churn rates (42.71%) compared to long-term contracts.

Two-year contract customers show extremely low churn (2.85%), indicating strong retention under long-term commitments.

This suggests that promoting long-term contracts may significantly reduce churn and stabilize recurring revenue.


### Churn by Internet Service

In [20]:
pd.read_sql("""
SELECT 
    InternetService,
    COUNT(*) AS total,
    SUM(CASE WHEN Churn='Yes' THEN 1 ELSE 0 END) AS churned,
    ROUND(
        100.0 * SUM(CASE WHEN Churn='Yes' THEN 1 ELSE 0 END) / COUNT(*), 2
    ) AS churn_rate
FROM customers
GROUP BY InternetService
ORDER BY churn_rate DESC;
""", conn)


,InternetService,total,churned,churn_rate
0,Fiber optic,3096,1297,41.89
1,DSL,2416,459,19.00
2,No,1520,113,7.43


Customers subscribed to fiber optic internet exhibit significantly higher churn rates (41.89%) compared to DSL (19%) and customers without internet service (7.43%).

This may indicate:
- Higher service dissatisfaction in fiber optic segment.
- Higher competition in high-speed internet market.
- Price sensitivity among premium customers.



In [21]:
pd.read_sql("""
SELECT
    CASE
        WHEN tenure <= 12 THEN '0-12 months'
        WHEN tenure <= 24 THEN '13-24 months'
        WHEN tenure <= 48 THEN '25-48 months'
        ELSE '48+ months'
    END AS tenure_group,
    COUNT(*) AS total_customers,
    SUM(CASE WHEN Churn='Yes' THEN 1 ELSE 0 END) AS churned,
    ROUND(
        100.0 * SUM(CASE WHEN Churn='Yes' THEN 1 ELSE 0 END) / COUNT(*), 2
    ) AS churn_rate
FROM customers
GROUP BY tenure_group
ORDER BY churn_rate DESC;
""", conn)


,tenure_group,total_customers,churned,churn_rate
0,0-12 months,2175,1037,47.68
1,13-24 months,1024,294,28.71
2,25-48 months,1594,325,20.39
3,48+ months,2239,213,9.51


Churn is heavily concentrated among early-stage customers.

Customers within their first 12 months exhibit a churn rate of 47.68%, nearly five times higher than customers with more than 48 months tenure (9.51%).

This suggests that early customer engagement and onboarding strategies are critical to retention.
